# <span style="color:teal">RBFE Network - Analysis</span>


In [1]:
# import libraries
import BioSimSpace as BSS
from scipy.stats import sem as sem
import sys
import os

if '/home/anna/Documents/cinnabar' not in sys.path:
    sys.path.insert(1, '/home/anna/Documents/cinnabar')
import cinnabar

print(cinnabar.__file__)

if '/home/anna/Documents/code/python' not in sys.path:
    sys.path.insert(1, '/home/anna/Documents/code/python')
import pipeline

print(pipeline.__file__)

from pipeline import *
from pipeline.utils import validate
from pipeline.analysis import *


Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.

/home/anna/Documents/cinnabar/cinnabar/__init__.py
/home/anna/Documents/code/python/pipeline/__init__.py


/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
/home/anna/Documents/code/python/pipeline/analysis/_plotting.py:422: DeprecationWarning: invalid escape sequence \D
  plt.ylabel("$\Delta\Delta$G$_{bind}$ / kcal$\cdot$mol$^{-1}$")
/home/anna/Documents/code/python/pipeline/analysis/_plotting.py:425: DeprecationWarning: invalid escape sequence \D
  plt.ylabel("$\Delta$G$_{bind}$ / kcal$\cdot$mol$^{-1}$")
/home/anna/Documents/code/python/pipeline/analysis/_plotting.py:579: DeprecationWarning: invalid escape sequence \D
  plt.ylabel("Computed $\Delta\Delta$G$_{bind}$ / kcal$\cdot$mol$^{-1}$")
/home/anna/Documents/code/python/pipeline/analysis/_pl

In [2]:
protein = "tyk2"

bench_folder = f"/home/anna/Documents/benchmark"
main_dir = f"{bench_folder}/extracted/{protein}"

# choose location for the files
net_file = f"{main_dir}/execution_model/network_lomap.dat"
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
# exp_file = f"{bench_folder}/inputs/experimental/tyk2.yml"
exp_file = f"/home/anna/Documents/benchmark/inputs/experimental/{protein}.yml"

if os.path.exists(f"{main_dir}/outputs_extracted/results"):
    results_folder = f"{main_dir}/outputs_extracted/results"
elif os.path.exists(f"{main_dir}/outputs/results"):
    results_folder = f"{main_dir}/outputs/results"
else:
    raise ValueError(f"results directory not found in the {main_dir}. please make sure results were written using the analysis script previously in the pipeline")

output_folder = validate.folder_path(f"{main_dir}/analysis", create=True)


In [3]:
all_analysis_object = analysis_network(results_folder,
                                       exp_file=exp_file,
                                       net_file=net_file,
                                       output_folder=output_folder,
                                       analysis_ext=ana_file
                                        )

# can add any other results files
# all_analysis_object.compute_other_results(file_name=None, name=None)
all_analysis_object.compute(cycle_closure=False)

could not convert analysis object lig_ejm31~lig_ejm48, GROMACS, pert, into dataframe for AMBER. Was it able to be computed earlier?
could not convert analysis object lig_jmc27~lig_jmc28, GROMACS, pert, into dataframe for AMBER. Was it able to be computed earlier?
could not convert analysis object lig_ejm46~lig_jmc28, GROMACS, pert, into dataframe for AMBER. Was it able to be computed earlier?
could not convert analysis object lig_ejm31~lig_ejm48, SOMD, pert, into dataframe for AMBER. Was it able to be computed earlier?
could not convert analysis object lig_ejm31~lig_ejm45, SOMD, pert, into dataframe for AMBER. Was it able to be computed earlier?
could not convert analysis object lig_jmc27~lig_jmc28, SOMD, pert, into dataframe for AMBER. Was it able to be computed earlier?
could not convert analysis object lig_ejm46~lig_jmc28, SOMD, pert, into dataframe for AMBER. Was it able to be computed earlier?
could not convert analysis object lig_ejm31~lig_ejm48, experimental, pert, into datafram

There are different options for plotting:
bar (pert or lig)
scatter (pert or lig) - can be plotted using cinnabar
eng vs eng
other results (eg from other workflows)
outliers
histograms (for the error of the runs, sem, legs)

In [ ]:
# bar
all_analysis_object.plot_bar_lig()
all_analysis_object.plot_bar_pert()

#scatter
all_analysis_object.plot_scatter_lig()
all_analysis_object.plot_scatter_pert()
all_analysis_object.plot_scatter_lig(use_cinnabar=True)
all_analysis_object.plot_scatter_pert(use_cinnabar=True)
for eng in all_analysis_object.engines:
    all_analysis_object.plot_scatter_lig(engine=eng)
    all_analysis_object.plot_scatter_pert(engine=eng)

    # outliers
    all_analysis_object.plot_outliers(engine=eng)
    all_analysis_object.plot_outliers(engine=eng, pert_val="val")

all_analysis_object.plot_histogram_legs(free_bound="free")
all_analysis_object.plot_histogram_legs(free_bound="bound")
all_analysis_object.plot_histogram_runs()
all_analysis_object.plot_histogram_sem()
all_analysis_object.plot_histogram_sem(pert_val="pert")

plot other results

In [ ]:
# write other results into a readable file format

other_results = sorted(glob.glob(f"{bench_folder}/inputs/other_computed/raw/*"))
for file in other_results:
    new_file = f"{file.replace(file.split('/')[-1], '')[:-1].replace('/raw','')}/{other_results.index(file)}_mcl1.csv"
    
    with open(new_file, "w") as write_file:
        writer = csv.writer(write_file, delimiter=",")

        writer.writerow(["lig_0", "lig_1", "freenrg",
                        "error", "engine", "analysis", "method"])
    
        with open(file, "r") as read_file:
            for line in read_file:
                if "Calculated" in line:
                    pass
                else:
                    lig_0 = line.split(',')[0].replace('protein_','')
                    lig_1 = line.split(',')[1].replace('protein_','')
                    freenrg = line.split(',')[2]
                    error = line.split(',')[3]

                writer.writerow([lig_0, lig_1, freenrg, error, "MBAR","other"])


In [ ]:
name = "shrodinger"
all_analysis_object.compute_other_results(sorted(glob.glob(f"{bench_folder}/inputs/other_computed/kuhn_2020/{protein}/*{name}.csv")), name=name)


In [ ]:
all_analysis_object._stats_object.compute_mue(pert_val="pert", y=name)

In [ ]:
all_analysis_object.plot_other_results(name=name, pert_val="pert")

In [ ]:
shared = []

for key in all_analysis_object.cinnabar_calc_pert_dict["AMBER"].keys():
    if key in all_analysis_object.cinnabar_calc_pert_dict["other"].keys():
        shared.append(key)

print(len(shared))
shared

plot just one engine to check

In [ ]:
all_analysis_object.plot_scatter_lig(engine="AMBER", use_cinnabar=True)
all_analysis_object.plot_scatter_pert(engine="AMBER", use_cinnabar=True)

plot engine vs engine

In [ ]:
all_analysis_object.plot_eng_vs_eng("SOMD","AMBER")

plot convergence

In [ ]:
ex_outputs_folder = f"{main_dir}/outputs_extracted"

converg_obj = plot_convergence(ex_outputs_folder,
                               perturbations=all_analysis_object.perturbations,
                               engines=all_analysis_object.engines,
                               file_ext=ana_file
                               )

converg_obj.plot_convergence_all()

remove a perturbation

In [ ]:
eng = "AMBER"
perturbations = ["lig_30~lig_40", "lig_27~lig_48"]
# ['lig_ejm42~lig_ejm44', 'lig_ejm42~lig_ejm49', 'lig_ejm42~lig_ejm45']
# ['lig_ejm42~lig_ejm49', 'lig_ejm45~lig_ejm55', 'lig_ejm43~lig_ejm50'] #gromacs
# ['lig_ejm45~lig_ejm53', 'lig_ejm44~lig_ejm45', 'lig_ejm31~lig_ejm49'] #somd

for pert in perturbations:
    try:
        all_analysis_object.remove_perturbations(pert)
    except:
        pass
    
print(all_analysis_object.perturbations)
all_analysis_object.compute()


# plot scatter plots
# all_analysis_object.plot_scatter_lig(engine=eng, use_cinnabar=True)
# all_analysis_object.plot_scatter_pert(engine="AMBER")
# all_analysis_object.plot_outliers(engine=eng, outliers=3, pert_val="pert")

statistical analysis vs exp, other results, each engine to the other etc

In [ ]:
all_analysis_object.calc_mae(pert_val="pert", engines=["GROMACS","AMBER"])
# all_analysis_object.calc_mae(pert_val="val")

Freenergworkflows analysis

In [7]:
all_analysis_object._get_exp_fwf(fwf_path="/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis")


({'lig_ejm31': (0.09554456211087381, 0.029),
  'lig_ejm42': (-0.14617801546097248, 0.019),
  'lig_ejm43': (1.3886502840592203, 0.25),
  'lig_ejm44': (2.239441725990767, 1.0),
  'lig_ejm45': (0.08299333578780074, 0.028),
  'lig_ejm46': (-1.6903948748417696, 0.0014),
  'lig_ejm47': (-0.05962612989820451, 0.022),
  'lig_ejm48': (0.6418015718625703, 0.072),
  'lig_ejm49': (1.9058204895277662, 0.6),
  'lig_ejm50': (0.6661380623268188, 0.075),
  'lig_ejm53': (1.7343155913659634, 0.45),
  'lig_ejm54': (-0.9024152875182272, 0.0054),
  'lig_ejm55': (0.43622104340109474, 0.051),
  'lig_jmc23': (-2.0792858601111703, 0.00075),
  'lig_jmc27': (-1.6542528257313989, 0.0015),
  'lig_jmc28': (-1.3497183935515489, 0.0026),
  'lig_jmc30': (-1.3090552793195833, 0.0027)},
 {'lig_ejm42~lig_ejm49': (2.0519985049887386, 0.6003007579538776),
  'lig_ejm31~lig_ejm49': (1.8102759274168925, 0.6007004245045945),
  'lig_ejm31~lig_ejm48': (0.5462570097516964, 0.07762087348130012),
  'lig_ejm31~lig_ejm53': (1.63877102

In [9]:
all_analysis_object._get_ana_fwf(fwf_path="/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis", engine="AMBER")
# all_analysis_object._get_stats_fwf(fwf_path="/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis", engine="AMBER")

KeyError: 'lig_2'

In [ ]:
# old

# import networkanalysis
# import experiments
# import stats

# exp_dicts = res_obj._get_exp_fwf(fwf_path='/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis/')
# for key in exp_dicts[0]:
#     print(f"{key} : {exp_dicts[0][key][0]}, {exp_dicts[0][key][1]}")

free_fwf_dict = res_obj._get_ana_fwf(fwf_path='/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis/', engine="SOMD")
for key in free_fwf_dict:
    print(f"{key} : {free_fwf_dict[key][0]}, {free_fwf_dict[key][1]}")

# fwf_stats = res_obj._get_stats_fwf(fwf_path='/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis/', engine="SOMD")
